In [56]:
P = 101

In [57]:
def modmul(a, b, m = P):
  return ((a % m) * (b % m)) % m

def mod_pow(a, b, m = P):
  if b == 0:
    return 1

  r = mod_pow(a, b//2, m)
  r = (r*r)%m

  if b%2:
    r = (r*a)%m
    
  return r


def get_positive(a, m=P):
    a = a % m
    a += m
    a = a % m
    return a

def moddiv(a, b, m = P):
  return modmul(a, mod_pow(b, m-2, m), m)

In [58]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def __eq__(self, p2):
    return self.x == p2.x and self.y == p2.y
  def __str__(self) -> str:
      return f"({self.x}, {self.y})"

In [59]:
class EllipticCurve:
  def __init__(self, a, b):
    self.a = a
    self.b = b
  
  def add(self, p1, p2, m = P):
    l = 0
    if p1 == p2:
      num = 3 * p1.x * p1.x + self.a
      den = 2 * p1.y
    else:
      num = p2.y - p1.y
      den = p2.x - p1.x
    
    l = moddiv(num, den, m)
    
    x3 = l * l - p1.x - p2.x
    y3 = l * (p1.x - x3) - p1.y
    
    x3 = get_positive(x3, m)
    y3 = get_positive(y3, m)
    
    return Point(x3, y3)

  def mul(self, k, p): # p is always generator point G
    while k != 1:
      p = self.add(p, p)
      k -= 1 
    return p
  
  def sub(self, p1, p2):
    np = Point(p2.x, -p2.y)
    return self.add(p1, np)

In [60]:
curve = EllipticCurve(2, 4) # Points lying on this curve:{0, 2}, {0, 5}, {1, 0}, {2, 3}, {2, 4}, {3, 3}, {3, 4}, {6, 1}, {6, 6}
G = Point(0, 2)

In [61]:
def encrypt(p, U):
  k = 5
  c = [
       curve.mul(k, G),
       curve.add(p, curve.mul(k, U))
  ]
  
  return c

In [62]:
def decrypt(C, R):
  p = curve.sub(C[1], curve.mul(R, C[0]))
  return p

In [63]:
R = 5               # Private key
U = curve.mul(R, G) # Public key

print("Private Key : ", R)
print("Public Key  : ", U)

Private Key :  5
Public Key  :  (64, 79)


In [64]:
plaintext = Point(3, 4)

print("Plaintext : ", plaintext)

Plaintext :  (3, 4)


In [65]:
ciphertext = encrypt(plaintext, U)

print("Cipher text : ", ciphertext[0], ciphertext[1])

Cipher text :  (64, 79) (17, 98)


In [66]:
p = decrypt(ciphertext, R)

# if((plaintext.x%P == p.x) or (plaintext.y%P == p.y)):
#     print(plaintext, p)

#     p = Point(plaintext.x, plaintext.y)

print("Decrypted Data : ", p)

Decrypted Data :  (3, 4)


In [67]:
assert(p == plaintext)

In [68]:
if(p==plaintext):
    print("p is same as plain text.")


p is same as plain text.
